In [1]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/NIST_Utils-master/pymetamap")
sys.path
#/home/tay/Documents/DSA-NIST

['',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python27.zip',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/plat-linux2',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-tk',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-old',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-dynload',
 '/home/tay/.local/lib/python2.7/site-packages',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages/IPython/extensions',
 '/home/tay/.ipython',
 '/home/tay/Documents/DSA-NIST/NIST_Utils-master/pymetamap']

In [2]:
import py_trees
import behaviours as be
from py_trees.blackboard import Blackboard
import pandas as pd
from scipy import spatial
from sklearn.cluster import KMeans
import numpy as np
from pandas import DataFrame
from annotator import annotator
import re
import matplotlib.pyplot as plt
import tqdm


In [3]:
import sys
sys.executable

'/home/tay/anaconda3/envs/py2_NIST/bin/python'

In [4]:
sys.path

['',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python27.zip',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/plat-linux2',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-tk',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-old',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/lib-dynload',
 '/home/tay/.local/lib/python2.7/site-packages',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages',
 '/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages/IPython/extensions',
 '/home/tay/.ipython',
 '/home/tay/Documents/DSA-NIST/NIST_Utils-master/pymetamap']

In [5]:
# read labeled cases
docu = './RAA_3658.xlsx'
df = pd.read_excel(docu)
su = 0
protocol = {}
narratives = df['Narrative']
vitals = df['Vitals']
inters = df['Interventions']
##TP##
narratives = narratives[0:100]
vitals = vitals[0:100]
inters = inters[0:100]
##TP##
gt = []
pts = set()
for item in df['HAYDON LABEL']:
    if not pd.isnull(item):
        gt.append(item)
        pts.add(item)

intermapping= {
    '12-lead ecg':['cardiac monitor'],
    'albuterol':['albuterol'],
    'aspirin':['aspirin'],
    'assist ventilation (bvm)':['bag valve mask ventilation'],
    'capnography (1) first reading':['capnography'],
    'capnography (2) seco':['capnography'],
    'capnography (2) second reading':['capnography'],
    'capnography (3) final reading':['capnography'],
    'cardiac monitor':['cardiac monitor'],
    'nasopharyngeal airway insertio':['nasopharyngeal airway'],
    'cpap':['cpap'],
    'dexamethasone (decadron)':['dexamethasone'],
    'dextrose 10%':['dextrose'],
    'dextrose 25%':['dextrose'],
    'dextrose 50%':['dextrose'],
    'dextrose 5% in 0.45% ns':['dextrose'],
    'duoneb':['albuterol','ipratropium'],
    'fentanyl':['fentanyl'],
    'glucagon':['glucagon'],
    'glutose':['oral glucose'],
    'hospital':['transport'],
    'hospital contact':['transport'],
    'intubation':['endotracheal tube'],
    'ipratropium (atrovent)':['ipratropium'],
    'iv':['normal saline'],
    'magnesium sulfate':['morphine sulfate'],
    'midazolam (versed)':['midazolam'],
    'naloxone (narcan)':['narcan'],
    'nitroglycerine':['nitroglycerine'],
    'normal saline':['normal saline'],
    'ondansetron (zofran)':['ondansetron'],
    'oral glucose':['oral glucose'],
    'oxygen':['oxygen'],
    'restraints':['physical restraint'],
    'suction':['suction the oropharynx','suction the nasopharynx'],
}
interventions = []
for item in inters:
    inter = item.strip('{}').split('}{')
    inter = [i.split(':')[-1].strip().lower() for i in inter]
    c_int = []
    for j in inter:
        if j in intermapping:
            c_int += intermapping[j]
        else:
            print j
    interventions.append(c_int)


oropharyngeal airway insertion
bleeding controlled
pulse oximetry
oropharyngeal airway insertion
oropharyngeal airway insertion
cpr
oropharyngeal airway insertion
oropharyngeal airway insertion
oropharyngeal airway insertion
spinal motion restriction
spinal motion restriction
adenosine (adenocard)
oropharyngeal airway insertion
oropharyngeal airway insertion


In [6]:
narratives[1:5]

1    D: Dispatched for back pain    A: Arrived to f...
2    D - Medic 490 was dispatched for a breathing p...
3    D- Dispatched to residence for Breathing Probl...
4    D: Dispatched for breathing problem priority 1...
Name: Narrative, dtype: object

In [9]:
from tqdm import tqdm_notebook as tqdm
EKGset = set()
pred_int = []
EKGdic = {
     '':'',
     'AV_Block_1st_Deg':'AV_Block-1st_Degree',
     'AV_Block_1st_Degree':'AV_Block-1st_Degree',
     'AV_Block_2nd_Degree_Type_1':'AV_Block_2nd_Degree_Type_1',
     'AV_Block_2nd_Degree_Type_2':'AV_Block_2nd_Degree_Type_2',
     'AV_Block_3rd_Degree':'AV_Block_3rd_Degree',
     'Asystole':'Asystole',
     'Artifact':'Artifact',
     'Atrial_Fibrill':'Atrial_Fibrillation',
     'Atrial_Fibrillation':'Atrial_Fibrillation',
     'Atrial_Flutter':'Atrial_Flutter',
     'Juncti':'Junctional_Rhythm',
     'Junctiona':'Junctional_Rhythm',
     'Junctional':'Junctional_Rhythm',
     'Other_(Not_Listed)':'Other_(Not_Listed)',
     'P': 'Paced_Rhythm',
     'PEA':'Pulseless_Electrical_Activity',
     'Pac':'Paced_Rhythm',
     'Paced_Rhythm':'Paced_Rhythm',
     'Premature_Ventricular_Contractions':'Premature_Ventricular_Contractions',
     'Right_Bundle_Branch_Block':'Right_Bundle_Branch_Block',
     'Left_Bundle_Branch_Block':'Left_Bundle_Branch_Block',
     'STEMI_Anterior_Ischemia':'STEMI_Anterior_Ischemia',
     'STEMI_Lateral_Ischemia':'STEMI_Lateral_Ischemia',
     'STEMI_Inferior_Ischemia':'STEMI_Inferior_Ischemia',
     'S':'Sinus_Rhythm',
     'Si':'Sinus_Rhythm',
     'Sin':'Sinus_Rhythm',
     'Sinu':'Sinus_Rhythm',
     'Sinus':'Sinus_Rhythm',
     'Sinus_':'Sinus_Rhythm',
     'Sinus_Arrhythmia':'Sinus_Arrhythmia',
     'Sinus_Bradycardia':'Sinus_Bradycardia',
     'Sinus_R':'Sinus_Rhythm',
     'Sinus_Rh':'Sinus_Rhythm',
     'Sinus_Rhy':'Sinus_Rhythm',
     'Sinus_Rhyth':'Sinus_Rhythm',
     'Sinus_Rhythm':'Sinus_Rhythm',
     'Sinus_Rhythm,Sinus_Tachycardia':'Sinus_Rhythm,Sinus_Tachycardia',
     'Sinus_T':'Sinus_Tachycardia',
     'Sinus_Tach':'Sinus_Tachycardia',
     'Sinus_Tachyc':'Sinus_Tachycardia',
     'Sinus_Tachycardi':'Sinus_Tachycardia',
     'Sinus_Tachycardia':'Sinus_Tachycardia',
     'Supravent':'Supraventricular_Tachycardia',
     'Supraventricular_Tachycardia':'Supraventricular_Tachycardia',
     'Ventricular_Fibrillation':'Ventricular_Fibrillation',
     'Ventricular_Tachycardia_(With_Pulse)':'Ventricular_Tachycardia',
     'Non_STEMI_Lateral_Ischemia':'Non_STEMI_Lateral_Ischemia'
}
# extract concept and calculate similarity
from ranking_func import rank
pool = set(['Medical - Abdominal Pain',
            'Medical - Altered Mental Status',
            'Medical - Seizure',
            'Medical - Respiratory Distress/Asthma/COPD/Croup/Reactive Airway',
            'General - Behavioral/Patient Restraint',
            'Medical - Overdose/Poisoning - Opioid',
            'Medical - Diabetic - Hypoglycemia',
            'Medical - Chest Pain - Cardiac Suspected'])

def pre_tick_handler(behaviour_tree):
    blackboard = Blackboard()
    blackboard.tick_num += 1
results = []
scores = []
ConceptRes = []
#for i,item in enumerate(tqdm(narratives)):
for i,item in enumerate((narratives)):
    if not pd.isnull(vitals[i]):
        vt = vitals[i].strip('{}').split('}{')
        vt = [it.split(':')[-1] for it in vt]
        for idx,it in enumerate(vt):
            if 'EKG-' in it:
                temp = it.split('EKG-')
                temp[1] = temp[1].replace(' ','_')
                temp[1] = temp[1].replace('-','_')
                if ',' in temp[1]:
                    t = temp[1].split(',')
                    t = [EKGdic[i] for i in t]
                    vt[idx] = temp[0] + 'EKG-' + ','.join(t)
                else:
                    vt[idx] = temp[0] + 'EKG-' + EKGdic[temp[1]]
                if len(temp[1]) > 0:
                    EKGset.add(temp[1])
        vt = [ite for l in vt for ite in l.strip().split(' ')]
        for idx in xrange(len(vt)):
            if idx < len(vt) and '-' not in vt[idx]:
                vt.pop(idx)
        for idx,it in enumerate(vt):
            temp = it.split('-')
            vt[idx] = (temp[0],temp[1])
    blackboard = Blackboard()
    blackboard.text = [item]
    root = py_trees.composites.Sequence("Root_1")
    IG = be.InformationGathering(inC = vt)
    TC = be.TextCollection()
    V = be.Vectorize()
    PS = be.ProtocolSelector()
    root.add_children([TC,IG,V,PS,be.protocols])
    behaviour_tree = py_trees.trees.BehaviourTree(root)
    behaviour_tree.add_pre_tick_handler(pre_tick_handler)
    behaviour_tree.setup(15)
    print('Item: ', i)
    behaviour_tree.tick_tock(
            sleep_ms=50,
            number_of_iterations=1,
            pre_tick_handler=None,
       post_tick_handler=None
        )
    scores.append(blackboard.ranking)
    ConceptRes.append(blackboard.ConcLog)
    results.append(blackboard.candi)
    res = []
    for key in blackboard.feedback:
        if blackboard.feedback[key] > 0.1:
            res.append((key, blackboard.feedback[key]))
    pred_int.append(res)
    

('Item: ', 0)
('Item: ', 1)
('Item: ', 2)
('Item: ', 3)
('Item: ', 4)
('Item: ', 5)
('Item: ', 6)


/home/tay/.local/lib/python2.7/site-packages/scipy/spatial/distance.py:702: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


('Item: ', 7)
('Item: ', 8)
('Item: ', 9)
('Item: ', 10)
('Item: ', 11)
('Item: ', 12)
('Item: ', 13)
('Item: ', 14)
('Item: ', 15)
('Item: ', 16)
('Item: ', 17)
('Item: ', 18)
('Item: ', 19)
('Item: ', 20)
('Item: ', 21)
('Item: ', 22)
('Item: ', 23)
('Item: ', 24)
('Item: ', 25)
('Item: ', 26)
('Item: ', 27)
('Item: ', 28)
('Item: ', 29)
('Item: ', 30)
('Item: ', 31)
('Item: ', 32)
('Item: ', 33)
('Item: ', 34)
('Item: ', 35)
('Item: ', 36)
('Item: ', 37)
('Item: ', 38)
('Item: ', 39)
('Item: ', 40)
('Item: ', 41)
('Item: ', 42)
('Item: ', 43)
('Item: ', 44)
('Item: ', 45)
('Item: ', 46)
('Item: ', 47)
('Item: ', 48)
('Item: ', 49)
('Item: ', 50)
('Item: ', 51)
('Item: ', 52)
('Item: ', 53)
('Item: ', 54)
('Item: ', 55)
('Item: ', 56)
('Item: ', 57)
('Item: ', 58)
('Item: ', 59)
('Item: ', 60)
('Item: ', 61)
('Item: ', 62)
('Item: ', 63)
('Item: ', 64)
('Item: ', 65)
('Item: ', 66)
('Item: ', 67)
('Item: ', 68)
('Item: ', 69)
('Item: ', 70)
('Item: ', 71)
('Item: ', 72)
('Item: ', 73

In [ ]:
#!conda upgrade notebook

In [ ]:
#conda install -n base -c conda-forge widgetsnbextension
#conda install -n py2_NIST -c conda-forge ipywidgets



In [10]:
pred_int_copy = pred_int[:]
pred_int = [[i[0] for i in case] for case in pred_int_copy]
pred_pos = [[i[1] for i in case] for case in pred_int_copy]

In [11]:
tTP = 0.
tFP = 0.
tFN = 0.
fpst = dict()
fnst = dict()
fps = []
fns = []
tlength = 0

precisions = []
recalls = []
f1scores = []
wprecisions = []
wrecalls = []
wf1scores = []
for i,item in enumerate(pred_int):
    TP = 0.
    FP = 0.
    FN = 0.
    fp = []
    fn = []
    l = len(interventions[i])
    tlength += l
    for m in item:
        if m in interventions[i]:
            TP += 1
            tTP += 1
        else:
            if m in fpst:
                fpst[m] += 1
            else:
                fpst[m] = 1
            FP += 1
            tFP += 1
            fp.append(m)
    for n in interventions[i]:
        if n not in pred_int[i]:
            if n in fnst:
                fnst[n] += 1
            else:
                fnst[n] = 1
            FN += 1
            tFN += 1
            fn.append(n)
    fps.append(fp)
    fns.append(fn)
    if TP == 0 and FP == 0:
        recalls.append(TP / (TP + FN))
        precisions.append(0.)
        f1scores.append(2 * TP / (2 * TP + FN + FP))
        wrecalls.append((TP / (TP + FN)) * l)
        wprecisions.append(0.)
        wf1scores.append((2 * TP / (2 * TP + FN + FP)) * l)
        continue
    wrecalls.append((TP / (TP + FN)) * l)
    wprecisions.append((TP / (TP + FP)) * l)
    wf1scores.append((2 * TP / (2 * TP + FN + FP)) * l)

In [12]:
interset = set()
for i in interventions:
    for j in i:
        interset.add(j)
int2idx = {}
for idx,item in enumerate(interset):
    int2idx[item] = idx
inter_vec = [[0] * len(interset) for _ in range(len(interventions))]
pre_int_vec = [[0] * len(interset) for _ in range(len(pred_int))]
for idx,case in enumerate(interventions):
    for inter in case:
        inter_vec[idx][int2idx[inter]] = 1
for idx,case in enumerate(pred_int):
    for inter in case:
        if inter in int2idx:
            pre_int_vec[idx][int2idx[inter]] = 1


In [13]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

print recall_score(np.array(inter_vec),np.array(pre_int_vec), average = 'weighted')
print recall_score(np.array(inter_vec),np.array(pre_int_vec), average = 'micro')
print precision_score(np.array(inter_vec),np.array(pre_int_vec), average = 'weighted')
print precision_score(np.array(inter_vec),np.array(pre_int_vec), average = 'micro')
print f1_score(np.array(inter_vec),np.array(pre_int_vec), average = 'weighted')
print f1_score(np.array(inter_vec),np.array(pre_int_vec), average = 'micro')

0.7009009009009008
0.7009009009009008
0.7514732765884841
0.9284009546539379
0.7230500311059501
0.7987679671457905


/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
fnst

{'albuterol': 1,
 'bag valve mask ventilation': 62,
 'capnography': 13,
 'cardiac monitor': 3,
 'ipratropium': 1,
 'midazolam': 2,
 'narcan': 28,
 'nasopharyngeal airway': 24,
 'normal saline': 6,
 'oxygen': 32,
 'physical restraint': 8,
 'transport': 2}

In [15]:
fpst

{'cardiac monitor': 3,
 'normal saline': 3,
 'oral glucose': 1,
 'oxygen': 19,
 'transport': 5}

In [16]:
df[0:5]

,Priority,CallType,ChiefComplaint,Impression,Vitals,Interventions,Narrative,Outcome,HAYDON LABEL
0,1.0,Abdominal Pain,{Poisoning/Overdose - Unintentional Med OD},{Environment - Poisoning/Drug Ingestion - PCR}...,{14:08:53: Pulse-99 Resp-8 BP-102/64 GCS-3 Glu...,{14:38:10: Hospital Contact}{14:38:16: IV}{14:...,A: Dispatched to the above address for an unco...,"Treated, Transported by EMS",Medical - Overdose/Poisoning - Opioid
1,1.0,Back Pain,{Poisoning/Overdose -Substance Abuse},{Abuse of Narcotic (i.e. Heroin) - PCR}{Neuro ...,{00:12:38: Pulse-98 Resp-8 BP-126/85 GCS-3 Glu...,{00:10:40: Assist Ventilation (BVM)}{00:10:41:...,D: Dispatched for back pain A: Arrived to f...,"Treated, Transported by EMS",Medical - Overdose/Poisoning - Opioid
2,1.0,Breathing Problems,{Poisoning/Overdose -Substance Abuse},{Abuse of Narcotic (i.e. Heroin) - PCR}{Weakne...,{17:35:24: Pulse-116 Resp-20 BP-72/51 GCS-13 G...,{17:36:00: Cardiac Monitor}{17:36:23: 12-Lead ...,D - Medic 490 was dispatched for a breathing p...,"Treated, Transported by EMS",Medical - Overdose/Poisoning - Opioid
3,1.0,Breathing Problems,{Altered Consciousness - Unresponsive}{Poisoni...,{Abuse of Narcotic (i.e. Heroin) - PCR}{Neuro ...,{13:13:30: Pulse-90 Resp-7 BP-0/0 GCS-3 Glucos...,{13:09:00: Assist Ventilation (BVM)}{13:09:00:...,D- Dispatched to residence for Breathing Probl...,"Treated, Transported by EMS",Medical - Overdose/Poisoning - Opioid
4,1.0,Breathing Problems,{Poisoning/Overdose - Toxic Inhalation},{Abuse of Alcohol - PCR}{Abuse of Narcotic (i....,{20:36:14: Pulse-101 Resp-16 BP-133/84 GCS-14 ...,{20:36:00: Cardiac Monitor}{20:40:38: IV}{20:4...,D: Dispatched for breathing problem priority 1...,"Treated, Transported by EMS",Medical - Overdose/Poisoning - Opioid


In [17]:
print(len(df))
print(len(df[0:100]))
print(len(fns))
fns[0:5]

3657
100
100


[['narcan'],
 ['bag valve mask ventilation'],
 ['oxygen'],
 ['bag valve mask ventilation',
  'oxygen',
  'capnography',
  'nasopharyngeal airway',
  'capnography',
  'capnography',
  'oxygen',
  'capnography'],
 ['nasopharyngeal airway', 'bag valve mask ventilation', 'oxygen']]

In [18]:
df_100 = df[0:100]

In [19]:
df_100['FN'] = fns
df_100['FP'] = fps
df_100['predictions'] = pred_int_copy
df_100['pre_protocols'] = results
df_100['similarities'] = scores

/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/tay/anaconda3/envs/py2_NIST/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [ ]:
df_100.to_excel('RAA_3657_RAA_3.xlsx',index = False)

In [20]:
bina = []
for i,r in enumerate(results):
    bina.append(gt[i] in r[:3])
print float(sum(bina))/len(bina)
        

0.84


In [21]:
sum(bina)

84

In [22]:
interventions1 = interventions[0:100]

In [ ]:
Inters = []
for item in interventions1:
    inter = item.strip('{}').split('}{')
    inter = [i.split(':')[-1].strip().lower() for i in inter]
    Inters.append(inter)
interset = set()
for i in Inters:
    for j in i:
        interset.add(j)

In [ ]:
FN2H = {
 '': 0,
 'aspirin': 1,
 'albuterol': 1,
 'bag valve mask ventilation': 3,
 'capnography': 1,
 'cardiac monitor': 1,
 'cpap': 3,
 'dexamethasone': 1,
 'dextrose': 3,
 'fentanyl': 0,
 'glucagon': 2,
 'ipratropium': 1,
 'midazolam': 3,
 'morphine sulfate': 0,
 'narcan': 3,
 'nasopharyngeal airway': 3,
 'nitroglycerine': 2,
 'normal saline': 0,
 'ondansetron': 0,
 'oral glucose': 1,
 'oxygen': 2,
 'physical restraint': 2,
 'transport': 0,
 'suction the oropharynx':2,
 'suction the nasopharynx':2
}

FP2H = {
 '': 0,
 'aspirin': 1,
 'bag valve mask ventilation': 1,
 'cardiac monitor': 0,
 'cpap': 2,
 'dexamethasone': 1,
 'dextrose': 1,
 'midazolam': 3,
 'narcan': 1,
 'nitroglycerin': 1,
 'normal saline': 1,
 'ondansetron': 0,
 'oral glucose': 1,
 'oxygen': 0,
 'physical restraint': 1,
 'transport': 0
}

In [ ]:
res = pd.read_excel('./RAA_3657.xlsx')

In [ ]:
hs = []
for i in res['FN']:
    sc = [FN2H[it.strip().strip('\'')] for it in i.strip('[]').split(',')]
    hs.append(max(sc))
for idx,i in enumerate(res['FP']):
    sc = [FP2H[it.strip().strip('\'')] for it in i.strip('[]').split(',')]
    if max(sc) > hs[idx]:
        hs[idx] = max(sc)


In [ ]:
print 'red:' + str(hs.count(3))
print 'orange:' + str(hs.count(2))
print 'yellow:' + str(hs.count(1))
print 'green:' + str(hs.count(0))

In [ ]:
from tqdm import tqdm_notebook as tqdm
EKGset = set()
pred_int = []
EKGdic = {
     '':'',
     'AV_Block_1st_Deg':'AV_Block-1st_Degree',
     'AV_Block_1st_Degree':'AV_Block-1st_Degree',
     'AV_Block_1st_Degree,Sinus_Rhythm':'AV_Block-1st_Degree,Sinus_Rhythm',
     'AV_Block_2nd_Degree_Type_1':'AV_Block_2nd_Degree_Type_1',
     'Asystole':'Asystole',
     'Artifact':'Artifact',
     'Atrial_Fibrill':'Atrial_Fibrillation',
     'Atrial_Fibrillation':'Atrial_Fibrillation',
     'Atrial_Flutter':'Atrial_Flutter',
     'Juncti':'Junctional_Rhythm',
     'Junctiona':'Junctional_Rhythm',
     'Junctional':'Junctional_Rhythm',
     'Other_(Not_Listed)':'Other_(Not_Listed)',
     'P': 'Paced_Rhythm',
     'PEA':'Pulseless_Electrical_Activity',
     'Pac':'Paced_Rhythm',
     'Paced_Rhythm':'Paced_Rhythm',
     'Premature_Ventricular_Contractions,,Sinus_Rhythm':'Premature_Ventricular_Contractions,Sinus_Rhythm',
     'Premature_Ventricular_Contractions,Sinus_Rhythm':'Premature_Ventricular_Contractions,Sinus_Rhythm',
     'Right_Bundle_Branch_Block':'Right_Bundle_Branch_Block',
     'STEMI_Anterior_Ischemia':'STEMI_Anterior_Ischemia',
     'STEMI_Lateral_Ischemia':'STEMI_Lateral_Ischemia',
     'S':'Sinus_Rhythm',
     'Si':'Sinus_Rhythm',
     'Sin':'Sinus_Rhythm',
     'Sinu':'Sinus_Rhythm',
     'Sinus':'Sinus_Rhythm',
     'Sinus_':'Sinus_Rhythm',
     'Sinus_Arrhythmia':'Sinus_Arrhythmia',
     'Sinus_Arrhythmia,Sinus_Rhythm':'Sinus_Arrhythmia,Sinus_Rhythm',
     'Sinus_Arrhythmia,Sinus_Tachycardia':'Sinus_Arrhythmia,Sinus_Tachycardia',
     'Sinus_Bradycardia':'Sinus_Bradycardia',
     'Sinus_R':'Sinus_Rhythm',
     'Sinus_Rh':'Sinus_Rhythm',
     'Sinus_Rhy':'Sinus_Rhythm',
     'Sinus_Rhyth':'Sinus_Rhythm',
     'Sinus_Rhythm':'Sinus_Rhythm',
     'Sinus_T':'Sinus_Tachycardia',
     'Sinus_Tach':'Sinus_Tachycardia',
     'Sinus_Tachyc':'Sinus_Tachycardia',
     'Sinus_Tachycardi':'Sinus_Tachycardia',
     'Sinus_Tachycardia':'Sinus_Tachycardia',
     'Supravent':'Supraventricular_Tachycardia',
     'Supraventricular_Tachycardia':'Supraventricular_Tachycardia',
     'Ventricular_Fibrillation':'Ventricular_Fibrillation',
     'Ventricular_Tachycardia_(With_Pulse)':'Ventricular_Tachycardia'
}
# extract concept and calculate similarity
from ranking_func import rank
pool = set(['Medical - Abdominal Pain',
            'Medical - Altered Mental Status',
            'Medical - Seizure',
            'Medical - Respiratory Distress/Asthma/COPD/Croup/Reactive Airway',
            'General - Behavioral/Patient Restraint',
            'Medical - Overdose/Poisoning - Opioid',
            'Medical - Diabetic - Hypoglycemia',
            'Medical - Chest Pain - Cardiac Suspected'])

def pre_tick_handler(behaviour_tree):
    blackboard = Blackboard()
    blackboard.tick_num += 1
results = []
scores = []
ConceptRes = []
for i,item in enumerate(tqdm(narratives)):
    vt = vitals[i].strip('{}').split('}{')
    vt = [it.split(':')[-1] for it in vt]
    for idx,it in enumerate(vt):
        if 'EKG-' in it:
            temp = it.split('EKG-')
            temp[1] = temp[1].replace(' ','_')
            temp[1] = temp[1].replace('-','_')
            vt[idx] = temp[0] + 'EKG-' + EKGdic[temp[1]]
            if len(temp[1]) > 0:
                EKGset.add(temp[1])
    vt = [ite for l in vt for ite in l.strip().split(' ')]
    for idx in xrange(len(vt)):
        if idx < len(vt) and '-' not in vt[idx]:
            vt.pop(idx)
    for idx,it in enumerate(vt):
        temp = it.split('-')
        vt[idx] = (temp[0],temp[1])
    blackboard = Blackboard()
    blackboard.text = item
    blackboard.case_num = i+1
    root = py_trees.composites.Sequence("Root_1")
    IG = be.IG(slist = './50candidateConcepts_EMS.txt', \
                                 inC = vt, neg_res = './all_results.txt')
    TC = be.TextCollection()
    V = be.Vectorize()
    PS = be.ProtocolSelector()
    root.add_children([TC,IG,V,PS,be.protocols])
    behaviour_tree = py_trees.trees.BehaviourTree(root)
    behaviour_tree.add_pre_tick_handler(pre_tick_handler)
    behaviour_tree.setup(15)
    behaviour_tree.tick_tock(
            sleep_ms=50,
            number_of_iterations=1,
            pre_tick_handler=None,
       post_tick_handler=None
        )
    scores.append(blackboard.ranking)
   # ConceptRes.append(blackboard.ConcLog)
    results.append(blackboard.candi)
    res = []
    for key in blackboard.feedback:
        if blackboard.feedback[key] > 0.1:
            res.append(key)
    pred_int.append(res)    

In [ ]:
TP = 0.
FP = 0.
FN = 0.
fpst = dict()
fnst = dict()
fps = []
fns = []

precisions = []
recalls = []
f1scores = []
for i,item in enumerate(pred_int):
    TP = 0.
    FP = 0.
    FN = 0.
    fp = []
    fn = []
    for m in item:
        if m in interventions[i]:
            TP += 1
        else:
            if m in fpst:
                fpst[m] += 1
            else:
                fpst[m] = 1
            FP += 1
            fp.append(m)
    for n in interventions[i]:
        if n not in pred_int[i]:
            if n in fnst:
                fnst[n] += 1
            else:
                fnst[n] = 1
            FN += 1
            fn.append(n)
    fps.append(fp)
    fns.append(fn)
    if TP == 0 and FP == 0:
        recalls.append(TP / (TP + FN))
        precisions.append(0.)
        f1scores.append(2 * TP / (2 * TP + FN + FP))
        continue
    recalls.append(TP / (TP + FN))
    precisions.append(TP / (TP + FP))
    f1scores.append(2 * TP / (2 * TP + FN + FP))

In [ ]:
ct = 0
fnc = 0
fpc = 0
for i in range(len(fns)):
    if len(fns[i]) == 0 and len(fps[i]) == 0:
        ct += 1
    if len(fns[i]) == 0:
        fpc += 1
    if len(fps[i]) == 0:
        fnc += 1
        
ct,fnc,fpc

In [ ]:
print 'M-precision:' + '%.3f'%(sum(precisions)/len(precisions))
print 'M-recall:' + '%.3f'%(sum(recalls)/len(recalls))
print 'M-F1-score:' + '%.3f'%(sum(f1scores)/len(f1scores))

In [ ]:
bina = []
for i,r in enumerate(results):
    bina.append(gt[i] in r[:3])
print float(sum(bina))/len(bina)

In [ ]:
df['FN'] = fns
df['FP'] = fps

In [ ]:
df.to_excel('RAA_166_25candidateConcepts_EMS_wU_RAA_3.xlsx',index = False)

In [ ]:
res = pd.read_excel('./RAA_286_Reduced_classes.xlsx')

In [ ]:
FN2H = {
 '': 0,
 'aspirin': 1,
 'albuterol': 1,
 'bag valve mask ventilation': 3,
 'capnography': 1,
 'cardiac monitor': 1,
 'cpap': 3,
 'dexamethasone': 1,
 'dextrose': 3,
 'fentanyl': 0,
 'glucagon': 2,
 'ipratropium': 1,
 'midazolam': 3,
 'morphine sulfate': 0,
 'narcan': 3,
 'nasopharyngeal airway': 3,
 'nitroglycerine': 2,
 'normal saline': 0,
 'ondansetron': 0,
 'oral glucose': 1,
 'oxygen': 2,
 'physical restraint': 2,
 'transport': 0,
 'suction the oropharynx':2,
 'suction the nasopharynx':2
}

FP2H = {
 '': 0,
 'aspirin': 1,
 'bag valve mask ventilation': 1,
 'cardiac monitor': 0,
 'cpap': 2,
 'dexamethasone': 1,
 'dextrose': 1,
 'midazolam': 3,
 'narcan': 1,
 'nitroglycerin': 1,
 'normal saline': 1,
 'ondansetron': 0,
 'oral glucose': 1,
 'oxygen': 0,
 'physical restraint': 1,
 'transport': 0
}

hs = []
for i in res['FN']:
    sc = [FN2H[it.strip().strip('\'')] for it in i.strip('[]').split(',')]
    hs.append(max(sc))
for idx,i in enumerate(res['FP']):
    sc = [FP2H[it.strip().strip('\'')] for it in i.strip('[]').split(',')]
    if max(sc) > hs[idx]:
        hs[idx] = max(sc)
        
print 'red:' + str(hs.count(3))
print 'orange:' + str(hs.count(2))
print 'yellow:' + str(hs.count(1))
print 'green:' + str(hs.count(0))